Linear optimization that require some of the variables to be integers are called Mixed Integer Programs (MIPs)<br>

OR-Tools provides two main tools for solving MIPs:<br>
 - MPSolver: A wrapper for several third-party MIP solvers, which use standard branch-and-bound techniques.<br>
 - CP-SAT solver: A constraint programming solver that uses SAT (satisfiability) methods.<br>

Task:<br>
Maximize x + 10y subject to the following constraints:<br>
x + 7 y ≤ 17.5<br>
x ≤ 3.5<br>
x ≥ 0<br>
y ≥ 0<br>
x, y integers<br>

In [1]:
from ortools.linear_solver import pywraplp

In [2]:
# Create the mip solver with the SCIP backend.
solver = pywraplp.Solver.CreateSolver('SCIP')

In [5]:
infinity = solver.infinity()
# x and y are integer non-negative variables.
x = solver.IntVar(0.0, infinity, 'x')
y = solver.IntVar(0.0, infinity, 'y')

print('Number of variables =', solver.NumVariables())

Number of variables = 2


In [6]:
# x + 7 * y <= 17.5.
solver.Add(x + 7 * y <= 17.5)

# x <= 3.5.
solver.Add(x <= 3.5)

print('Number of constraints =', solver.NumConstraints())

Number of constraints = 2


In [7]:
# Maximize x + 10 * y.
solver.Maximize(x + 10 * y)

In [8]:
#Call status
status = solver.Solve()

In [9]:
if status == pywraplp.Solver.OPTIMAL:
    print('Solution:')
    print('Objective value =', solver.Objective().Value())
    print('x =', x.solution_value())
    print('y =', y.solution_value())
else:
    print('The problem does not have an optimal solution.')

Solution:
Objective value = 23.0
x = 3.0
y = 2.0


# Using Arrays to Define a Model

Maximize 7x1 + 8x2 + 2x3 + 9x4 + 6x5 subject to the following constraints:<br>
5x1	+	7x2	+	9x3	+	2x4	+	1x5	≤ 250<br>
18x1	+	4x2	-	9x3	+	10x4 + 12x5	≤ 285<br>
4x1	+	7x2	+	3x3	+	8x4	+	5x5	≤ 211<br>
5x1	+	13x2	+	16x3	+	3x4	- 7x5 ≤ 315<br>
where x1, x2, ..., x5 are non-negative integers.<br>

In [10]:
def create_data_model():
  """Stores the data for the problem."""
  data = {}
  data['constraint_coeffs'] = [
      [5, 7, 9, 2, 1],
      [18, 4, -9, 10, 12],
      [4, 7, 3, 8, 5],
      [5, 13, 16, 3, -7],
  ]
  data['bounds'] = [250, 285, 211, 315]
  data['obj_coeffs'] = [7, 8, 2, 9, 6]
  data['num_vars'] = 5
  data['num_constraints'] = 4
  return data


def main():
  data = create_data_model()
  # Create the mip solver with the SCIP backend.
  solver = pywraplp.Solver.CreateSolver('SCIP')
  infinity = solver.infinity()
  x = {}
  for j in range(data['num_vars']):
    x[j] = solver.IntVar(0, infinity, 'x[%i]' % j)
  print('Number of variables =', solver.NumVariables())

  for i in range(data['num_constraints']):
    constraint = solver.RowConstraint(0, data['bounds'][i], '')
    for j in range(data['num_vars']):
      constraint.SetCoefficient(x[j], data['constraint_coeffs'][i][j])
  print('Number of constraints =', solver.NumConstraints())
  # In Python, you can also set the constraints as follows.
  # for i in range(data['num_constraints']):
  #  constraint_expr = \
  # [data['constraint_coeffs'][i][j] * x[j] for j in range(data['num_vars'])]
  #  solver.Add(sum(constraint_expr) <= data['bounds'][i])

  objective = solver.Objective()
  for j in range(data['num_vars']):
    objective.SetCoefficient(x[j], data['obj_coeffs'][j])
  objective.SetMaximization()
  # In Python, you can also set the objective as follows.
  # obj_expr = [data['obj_coeffs'][j] * x[j] for j in range(data['num_vars'])]
  # solver.Maximize(solver.Sum(obj_expr))

  status = solver.Solve()

  if status == pywraplp.Solver.OPTIMAL:
    print('Objective value =', solver.Objective().Value())
    for j in range(data['num_vars']):
      print(x[j].name(), ' = ', x[j].solution_value())
    print()
    print('Problem solved in %f milliseconds' % solver.wall_time())
    print('Problem solved in %d iterations' % solver.iterations())
    print('Problem solved in %d branch-and-bound nodes' % solver.nodes())
  else:
    print('The problem does not have an optimal solution.')

In [11]:
main()

Number of variables = 5
Number of constraints = 4
Objective value = 260.0000000000005
x[0]  =  8.0
x[1]  =  21.0
x[2]  =  0.0
x[3]  =  2.0
x[4]  =  3.0

Problem solved in 28.000000 milliseconds
Problem solved in 54 iterations
Problem solved in 7 branch-and-bound nodes
